In [572]:
import Radar as R
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import os
import time
import argparse
import pandas as pd
import time
from pyquaternion import Quaternion
from numpy import genfromtxt
import os
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN
from sklearn import metrics
import pandas as pd
from pyquaternion import Quaternion
from numpy.linalg import norm
import math
import importlib

In [573]:
importlib.reload(R)

<module 'Radar' from '/Users/shannonholmes/Desktop/Python Programs/AWN/Radar.py'>

In [574]:
frame_num = '1'
Frame_time = 0
## initialization ##
Cluster_dict = {}
Next_Cluster_dict = {}
Next_Velocity_dict = {}
## KalmanFilter for Client 1
KalmanMeasurements = np.zeros((4,1))
KalmanP = np.zeros((4,4,1))
Innovation = np.zeros((2,1))
KalmanF = np.zeros((4,2,1))
ConditionalX = np.zeros((4,1))
ConditionalP=np.identity(4)

## KalmanFilter for Client 2 

KalmanMeasurements2 = np.zeros((4,1))
KalmanP2 = np.zeros((4,4,1))
Innovation2 = np.zeros((2,1))
KalmanF2 = np.zeros((4,2,1))
ConditionalX2 = np.zeros((4,1))
ConditionalP2 = np.identity(4)

x = 0
Microframe = 0
client1 = R.client(0)
client2 = R.client(1)

Start_time = time.time()

In [575]:
data_df = pd.read_csv('/Users/shannonholmes/Desktop/Python Programs/AWN/test2.csv')
num_frames = data_df.iloc[:,1].nunique()
print("DEBUG: Number of unique frames: ", num_frames)

DEBUG: Number of unique frames:  50


In [576]:
""" data = np.array([])

# Select rows with matching frame number if 1 then get the frame number from the first row in the df
if frame_num == 1:
    frame_num = data_df.iloc[0,1]

selected_rows = data_df[data_df.iloc[:, 1] == frame_num]

# Convert selected rows to numpy array
data = np.array(selected_rows)
print(selected_rows.index[-1])
# Get the next frame number
next_frame_num = data_df.iloc[selected_rows.index[-1]+1, 1]
print(next_frame_num) ##TODO verify this is working##

data = np.reshape(data, (int(len(data)/8), 8))
data = data.astype(float) """

data = data_df[0:1200]
print(data)

      0.101970411  13007     0  0.476763  1.92219  -1.86371  -0.217266   2238
0         0.10197  13007     1  0.520106  1.91091 -1.863710  -0.217266   2231
1         0.10197  13007     2  0.476763  1.96328 -1.820370  -0.217266   2282
2         0.10197  13007     3  0.520106  1.95225 -1.820370  -0.217266   2277
3         0.10197  13007     4  0.476763  2.00260 -1.777030  -0.217266   2241
4         0.10197  13007     5  0.520106  1.99178 -1.777030  -0.217266   2214
...           ...    ...   ...       ...      ...       ...        ...    ...
1195      0.10197  13007  1196  0.677506  4.61344 -0.752784   0.000000  43513
1196      0.10197  13007  1197  0.225835  4.66899 -0.677506   0.000000  49086
1197      0.10197  13007  1198  0.301114  4.66474 -0.677506   0.000000  63062
1198      0.10197  13007  1199  0.376392  4.65927 -0.677506   0.000000  71490
1199      0.10197  13007  1200  0.451671  4.65258 -0.677506   0.000000  71066

[1200 rows x 8 columns]


In [577]:
if Microframe == 0:
    Frame = R.rframe(frame_num,data,None,None,Start_time)
else:
    Frame = R.rframe(frame_num,data,None,Prev_Frame,Start_time)

data2 = Frame.data
data2 = data2.iloc[:,3:6]
db = DBSCAN(eps=0.3, min_samples=25).fit(data2)
labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)  # Number of Clusters, Label 0-N
core_samples_mask = np.zeros_like(labels, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

print(labels)
print(n_clusters_)

[0 0 0 ... 2 2 2]
4


In [578]:
CorePoints = np.zeros((n_clusters_,3))
CoreNum = np.zeros((n_clusters_))
CoreSum = np.zeros((n_clusters_,3))

for point in range(0,len(Frame.data)):
    if labels[point] != -1:
        CoreNum[labels[point]] = CoreNum[labels[point]] + 1
        CoreSum[labels[point]][0] = CoreSum[labels[point]][0] + Frame.data.loc[point].iat[3]
        CoreSum[labels[point]][1] = CoreSum[labels[point]][1] + Frame.data.loc[point].iat[4]
        CoreSum[labels[point]][2] = CoreSum[labels[point]][2] + Frame.data.loc[point].iat[5]

for corepoint_id in range(0,n_clusters_):
    CorePoints[corepoint_id] = CoreSum[corepoint_id]/CoreNum[corepoint_id]

In [579]:
Cluster_dict = Next_Cluster_dict.copy()

Next_Cluster_dict = {}

if Cluster_dict:
    if len(Cluster_dict) >= n_clusters_:
        for corepoint_id in range (0,n_clusters_):
            Min_dis = 100
            Min_key = 100
            for keys in Cluster_dict:
                dis = (Cluster_dict[keys][0]-CorePoints[corepoint_id][0])**2+(Cluster_dict[keys][1]-CorePoints[corepoint_id][1])**2\
                        +(Cluster_dict[keys][2]-CorePoints[corepoint_id][2])**2
                if dis < Min_dis:
                    Min_key = keys
                    Min_dis = dis
            Next_Cluster_dict[Min_key] = CorePoints[corepoint_id]
    elif len(Cluster_dict) < n_clusters_:
        for keys in Cluster_dict:
            Min_dis = 100
            Min_core_id = 100
            for corepoint_id in range(0,n_clusters_):
                dis = (Cluster_dict[keys][0]-CorePoints[corepoint_id][0])**2+(Cluster_dict[keys][1]-CorePoints[corepoint_id][1])**2\
                        +(Cluster_dict[keys][2]-CorePoints[corepoint_id][2])**2
                if dis < Min_dis:
                    Min_core_id = corepoint_id
                    Min_dis = dis
            Next_Cluster_dict[keys] = CorePoints[Min_core_id]

    for keys in Cluster_dict:
        if keys in Next_Cluster_dict:
            Next_Velocity_dict[keys] = [(Next_Cluster_dict[keys][0] - Cluster_dict[keys][0]) / 0.18,
                                    (Next_Cluster_dict[keys][1] - Cluster_dict[keys][1]) / 0.18,
                                    (Next_Cluster_dict[keys][2] - Cluster_dict[keys][2]) / 0.18]
    
else:
    for corepoint_id in range(0,n_clusters_):
        Next_Cluster_dict[corepoint_id] = CorePoints[corepoint_id]

print(Next_Cluster_dict)

{0: array([ 0.60522164,  2.13589176, -0.9105395 ]), 1: array([ 1.57118226,  5.98173742, -0.45831769]), 2: array([ 0.47821276,  4.62249707, -0.88735821]), 3: array([0.58591127, 3.29079695, 1.81936381])}


In [580]:
R.trackOrientation6D(Frame_time,client1)
R.trackOrientation6D(Frame_time,client2)
R.getVelocity(Frame_time,client1,client2)